In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

# Ruta del archivo CSV
file_path = '/Users/diegogloriasalamanca/Library/Mobile Documents/com~apple~CloudDocs/Master Data Science/99 TFM/TFM Inmobiliario/datamunging/consolidated_data.csv'

# Leer el archivo CSV en un DataFrame
df = pd.read_csv(file_path)

# Verificar el tamaño del DataFrame
print(f"Tamaño del DataFrame: {df.shape}")

# Establecer valores NaN para ciertas columnas
puntos_interes = ['cinema', 'restaurant', 'theatre', 'police', 'supermarket',
                   'pharmacy', 'bar', 'playground', 'cafe', 'monument', 'station',
                   'clinic', 'post_office', 'convenience', 'sports_centre', 'atm',
                   'memorial', 'museum', 'artwork', 'bicycle_rental', 'library', 
                   'waste_disposal', 'platform', 'viewpoint', 'school', 'fuel', 'works', 
                   'bus_station', 'clothes', 'fire_station', 'college', 'hospital', 
                   'doctors', 'mall', 'stop_position', 'park', 'prison', 'fast_food', 
                   'garden', 'marketplace', 'fountain', 'townhall', 'industrial', 
                   'ferry_terminal', 'public_building', 'place_of_worship', 'bench', 
                   'drinking_water', 'gym', 'stop_area', 'bird_hide', 'food_court', 'shelter', 
                   'information', 'stationery', 'farm', 'yoga', 'attraction', 'mobile_library', 
                   'chalet', 'parking_entrance', 'parking', 'wholesale', 'bakery', 'pub', 
                   'vending_machine', 'pastry', 'yes', 'sailing_school', 'windsurfing_rental', 
                   'ice_cream', 'internet_cafe', 'clock', 'dentist']

df.loc[:, puntos_interes] = np.nan

# Eliminar filas donde 'precio' es NaN, 0, o infinito
df = df[~df['precio'].isin([0, np.inf, -np.inf]) & df['precio'].notna()]

# Separar características y variable objetivo
X = df.drop('precio', axis=1)
y = df['precio']

# Imputar valores NaN en y con la mediana
y = y.fillna(y.median())

# Identificar columnas numéricas
num_cols = X.select_dtypes(include=['int64', 'float64']).columns

# Preprocesamiento para columnas numéricas
num_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value=0)),
    ('scaler', StandardScaler())
])

# Combinar preprocesadores solo para columnas numéricas
preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_transformer, num_cols)
    ])

# Crear pipelines para RandomForest y LinearRegression
rf_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(n_estimators=100, random_state=42))
])

lr_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

# Dividir el conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X[num_cols], y, test_size=0.2, random_state=42)

# Verificar tamaños de conjuntos de entrenamiento y prueba
print(f"Tamaño de X_train: {X_train.shape}")
print(f"Tamaño de X_test: {X_test.shape}")

# Entrenar y evaluar el modelo de RandomForest
rf_pipeline.fit(X_train, y_train)
y_pred_rf = rf_pipeline.predict(X_test)

mae_rf = mean_absolute_error(y_test, y_pred_rf)
mse_rf = mean_squared_error(y_test, y_pred_rf)
rmse_rf = np.sqrt(mse_rf)
r2_rf = r2_score(y_test, y_pred_rf)

print("Resultados de RandomForest:")
print(f'MAE: {mae_rf}')
print(f'RMSE: {rmse_rf}')
print(f'R2 Score: {r2_rf}')

# Entrenar y evaluar el modelo de LinearRegression
lr_pipeline.fit(X_train, y_train)
y_pred_lr = lr_pipeline.predict(X_test)

mae_lr = mean_absolute_error(y_test, y_pred_lr)
mse_lr = mean_squared_error(y_test, y_pred_lr)
rmse_lr = np.sqrt(mse_lr)
r2_lr = r2_score(y_test, y_pred_lr)

print("\nResultados de LinearRegression:")
print(f'MAE: {mae_lr}')
print(f'RMSE: {rmse_lr}')
print(f'R2 Score: {r2_lr}')

# Imprimir algunas predicciones para verificar
print("\nAlgunas predicciones de RandomForest:")
for i in range(5):
    print(f"Precio real: {y_test.iloc[i]}, Precio predicho: {y_pred_rf[i]}")

print("\nAlgunas predicciones de LinearRegression:")
for i in range(5):
    print(f"Precio real: {y_test.iloc[i]}, Precio predicho: {y_pred_lr[i]}")

# Obtener importancia de características de RandomForest
rf_model = rf_pipeline.named_steps['regressor']
importances = rf_model.feature_importances_
feature_names = rf_pipeline.named_steps['preprocessor'].get_feature_names_out()

# Crear un DataFrame con la importancia de características
importance_df = pd.DataFrame({'feature': feature_names, 'importance': importances})
importance_df = importance_df.sort_values('importance', ascending=False)

print("\nCaracterísticas más importantes de RandomForest:")
print(importance_df.head(10))

print("Modelos completados.")

/var/folders/gk/kfm0w04x18d1tr94k7ml70yc0000gn/T/ipykernel_2859/2122279516.py:16: DtypeWarning: Columns (11,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


Tamaño del DataFrame: (175700, 18)
Tamaño de X_train: (138977, 81)
Tamaño de X_test: (34745, 81)
Resultados de RandomForest:
MAE: 138323.9412119245
RMSE: 349177.3081738556
R2 Score: 0.6995875316913116

Resultados de LinearRegression:
MAE: 291887.3504135291
RMSE: 596223.0724226853
R2 Score: 0.12412226883095967

Algunas predicciones de RandomForest:
Precio real: 370.0, Precio predicho: 1178.8780541279364
Precio real: 1000.0, Precio predicho: 65833.00675
Precio real: 3250.0, Precio predicho: 88803.5
Precio real: 125000.0, Precio predicho: 111293.81666666667
Precio real: 185000.0, Precio predicho: 207340.16666666663

Algunas predicciones de LinearRegression:
Precio real: 370.0, Precio predicho: 62700.51452012919
Precio real: 1000.0, Precio predicho: 170210.71005283803
Precio real: 3250.0, Precio predicho: 493026.25693943276
Precio real: 125000.0, Precio predicho: 156394.7074068054
Precio real: 185000.0, Precio predicho: 248543.46398226486

Características más importantes de RandomForest:
 